In [1]:
import pandas as pd
import numpy as np
import json
import random

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import rdBase
rdBase.DisableLog('rdApp.error') 

from dotenv import load_dotenv
load_dotenv()

from mol_level_model.model import DescriptorContrastiveModel
from mol_level_model.utils import calculate_mean_and_std, approximate_homo_lumo

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(parent_dir)

from model.model import GNNFingerprint3D
from hypotheses.compare_with_other_fingerprints.utils import smiles_to_3D
from data_preprocessing.descriptors import calculate_descriptors_v2

In [2]:
data_path = os.getenv("DATA_PATH")
models_path = os.getenv("MODELS_PATH")

columns = ["mu", "zpve", "energy_U0"]

In [3]:
from rdkit.Geometry import Point3D
from rdkit.Chem import Conformer

def attach_3D_coords_to_mol(mol, coords):
    """
    Assigns 3D coordinates (list of [atom_symbol, x, y, z]) to mol.
    """
    conf = Conformer(mol.GetNumAtoms())
    for i, (symbol, x, y, z) in enumerate(coords):
        conf.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))

    mol.AddConformer(conf, assignId=True)
    return mol


In [4]:
def get_descriptor(smiles, model, scaler, device, conf=None, homo=None, lumo=None):
    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = Chem.AddHs(mol)
    except:
        return torch.full((1024,), float('nan')).to("cuda")
     
    if conf:
        c = Conformer(mol.GetNumAtoms())
        for i, (symbol, x, y, z) in enumerate(conf):
            c.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))
        mol.AddConformer(c, assignId=True)
    else:
        result = AllChem.EmbedMolecule(mol, randomSeed=42)
        if result != 0: return torch.full((1024,), float('nan')).to(device)
        mol = Chem.RemoveHs(mol)

    if not homo:
        homo, lumo = approximate_homo_lumo(mol)

    desc = calculate_descriptors_v2(smiles, mol, homo, lumo, False)
    desc.pop("SMILES", None)

    for key in desc:
        desc[key] = (desc[key] - scaler[key]['mean']) / scaler[key]['std']

    rec = torch.tensor(list(desc.values()), dtype=torch.float32).unsqueeze(0).to(device)
    pred = model(rec)
    return pred.squeeze(0)

### 2D TASK

In [5]:
def read_data(path, ki_threshold):
    df = pd.read_csv(path, sep=";")
    df['Activity'] = df['Standard Value'].apply(lambda x: 1 if x < ki_threshold else 0)

    df = df[['Smiles', 'Activity']].dropna()

    X_train, X_test, y_train, y_test = train_test_split(
        df['Smiles'], df['Activity'], test_size=0.2, random_state=42
    )

    return X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)

In [6]:
def get_score(X_train, y_train, X_test, y_test, task = "c"):
    s_scaler = StandardScaler()
    X_train = s_scaler.fit_transform(X_train)
    X_test = s_scaler.transform(X_test)

    # PCA to 167 dim
    pca = PCA(n_components=167)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    if task == "c":
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        print("Classification Report:")
        print(classification_report(y_test, y_pred))

    else:
        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        print("Regression Metrics:")
        print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
        print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
        print(f"R²:   {r2_score(y_test, y_pred):.4f}")

In [19]:
fingerprint_model_3D = GNNFingerprint3D(13, 5)
fingerprint_model_3D.load_state_dict(torch.load(os.path.join(models_path, "FINAL_GNN.pth")))
fingerprint_model_3D = fingerprint_model_3D.to("cuda")
fingerprint_model_3D.eval()

fingerprint_model_2D = GNNFingerprint3D(13, 5)
fingerprint_model_2D.load_state_dict(torch.load(os.path.join(models_path, "GNN_MORE_WEIGHT_3D.pth")))
fingerprint_model_2D = fingerprint_model_2D.to("cuda")
fingerprint_model_2D.eval()

descriptor_model = DescriptorContrastiveModel(input_dim=18)
descriptor_model.load_state_dict(torch.load(os.path.join(models_path, "DESCRIPTOR_LEVEL_MODEL.pth")))
descriptor_model = descriptor_model.to("cuda")
descriptor_model.eval()

with open(os.path.join(data_path, "means_and_stds.json")) as f:
    scaler = json.load(f)

desc_scaler = calculate_mean_and_std(os.path.join(data_path, "qm9_data"))

C:\Users\jakub\AppData\Local\Temp\ipykernel_17792\753072789.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fingerprint_model_3D.load_state_dict(torch.load(os.path.join(

In [10]:
folder = os.path.join(data_path, "CHEMBL")
device = "cuda" if torch.cuda.is_available() else "cpu"

for db in os.listdir(folder):
    print("============================================")
    print(db)
    print("============================================")

    data = os.path.join(folder, db)
    X_train, X_test, y_train, y_test = read_data(data, 100)

    X_train, y_train = list(X_train), list(y_train)
    X_test, y_test = list(X_test), list(y_test)
    
    ### Descriptor
    print("****************")
    print("Mol level model")
    print("****************")
    X_train_desc = np.array([get_descriptor(smiles, descriptor_model,desc_scaler, device).detach().cpu() for smiles in X_train])
    X_test_desc = np.array([get_descriptor(smiles, descriptor_model,desc_scaler, device).detach().cpu() for smiles in X_test])

    rows_all_nan_train = np.isnan(X_train_desc).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_desc).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_desc = np.delete(X_train_desc, nan_indices_train, axis=0)
    y_train_desc = np.delete(y_train, nan_indices_train, axis=0)
    X_test_desc = np.delete(X_test_desc, nan_indices_test, axis=0)
    y_test_desc = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_desc, y_train_desc, X_test_desc, y_test_desc)

    ### 3D
    print("****************")
    print("Atom level model")
    print("*****************")

    X_train_3D = np.array([smiles_to_3D(smiles, fingerprint_model_2D, scaler, False).detach().cpu() for smiles in X_train])
    X_test_3D = np.array([smiles_to_3D(smiles, fingerprint_model_2D, scaler, False).detach().cpu() for smiles in X_test])

    rows_all_nan_train = np.isnan(X_train_3D).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_3D).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_3D = np.delete(X_train_3D, nan_indices_train, axis=0)
    y_train_3D = np.delete(y_train, nan_indices_train, axis=0)
    X_test_3D = np.delete(X_test_3D, nan_indices_test, axis=0)
    y_test_3D = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_3D, y_train_3D, X_test_3D, y_test_3D)

CHEMBL1833_5HT2B.csv
****************
Mol level model
****************
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.97      0.90       408
           1       0.85      0.45      0.59       134

    accuracy                           0.84       542
   macro avg       0.84      0.71      0.74       542
weighted avg       0.84      0.84      0.82       542

****************
Atom level model
*****************
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       408
           1       0.86      0.47      0.61       134

    accuracy                           0.85       542
   macro avg       0.86      0.72      0.76       542
weighted avg       0.85      0.85      0.83       542

CHEMBL214_5HT1A.csv
****************
Mol level model
****************


[10:42:30] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[10:42:30] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[10:44:16] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[10:44:16] UFFTYPER: Warning: hybridization set to SP3 for atom 29


Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.64      0.68       549
           1       0.73      0.81      0.77       674

    accuracy                           0.73      1223
   macro avg       0.73      0.72      0.73      1223
weighted avg       0.73      0.73      0.73      1223

****************
Atom level model
*****************


[10:51:24] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[10:51:24] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[10:51:24] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[10:54:55] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[10:54:55] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[10:54:55] UFFTYPER: Warning: hybridization set to SP3 for atom 29


Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.70      0.73       549
           1       0.77      0.81      0.79       673

    accuracy                           0.76      1222
   macro avg       0.76      0.76      0.76      1222
weighted avg       0.76      0.76      0.76      1222

CHEMBL224_5HT2A.csv
****************
Mol level model
****************
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       704
           1       0.84      0.67      0.74       532

    accuracy                           0.80      1236
   macro avg       0.81      0.79      0.79      1236
weighted avg       0.81      0.80      0.80      1236

****************
Atom level model
*****************
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       704
           1       0.89      0.72      0.80   

### 3D TASK

In [20]:
def read_data_3D(db_name, size):
    path = os.path.join(data_path, db_name)
    all_files = [ff for f in os.listdir(path) if os.path.isfile(ff := os.path.join(path, f))]

    random_files = random.sample(all_files, size)
    l = []
    for i, file in enumerate(random_files):
        with open(file, 'r') as f:
            data = json.load(f)
        rec = {"smiles": data["smiles"], "conf": data["atoms"], "homo": data["homo"], "lumo": data["lumo"]}
        for col in columns:
            rec[col] = data[col]
        l.append(rec)

    return pd.DataFrame(l)

In [21]:
df = read_data_3D("qm9_data_json", 20000)
device = "cuda" if torch.cuda.is_available() else "cpu"

for col in columns:
    print("============================================")
    print(col)
    print("============================================")

    X = df[["smiles", "conf", "homo", "lumo"]]
    y = df[col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    
    ### Descriptor
    print("****************")
    print("Mol level model")
    print("****************")
    X_train_desc = np.array([get_descriptor(mol[0], descriptor_model, desc_scaler, device, mol[1], mol[2], mol[3]).detach().cpu() for mol in X_train])
    X_test_desc = np.array([get_descriptor(mol[0], descriptor_model, desc_scaler, device, mol[1], mol[2], mol[3]).detach().cpu() for mol in X_test])
    
    rows_all_nan_train = np.isnan(X_train_desc).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_desc).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_desc = np.delete(X_train_desc, nan_indices_train, axis=0)
    y_train_desc = np.delete(y_train, nan_indices_train, axis=0)
    X_test_desc = np.delete(X_test_desc, nan_indices_test, axis=0)
    y_test_desc = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_desc, y_train_desc, X_test_desc, y_test_desc, "r")

    ### 3D
    print("****************")
    print("Atom level model")
    print("*****************")

    X_train_3D = np.array([smiles_to_3D(smiles, fingerprint_model_3D, scaler).detach().cpu() for smiles in X_train])
    X_test_3D = np.array([smiles_to_3D(smiles, fingerprint_model_3D, scaler).detach().cpu() for smiles in X_test])

    rows_all_nan_train = np.isnan(X_train_3D).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_3D).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_3D = np.delete(X_train_3D, nan_indices_train, axis=0)
    y_train_3D = np.delete(y_train, nan_indices_train, axis=0)
    X_test_3D = np.delete(X_test_3D, nan_indices_test, axis=0)
    y_test_3D = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_3D, y_train_3D, X_test_3D, y_test_3D, "r")

mu
****************
Mol level model
****************
Regression Metrics:
MAE:  0.1032
RMSE: 0.5609
R²:   -0.0657
****************
Atom level model
*****************
Regression Metrics:
MAE:  0.1492
RMSE: 0.2919
R²:   0.7113
zpve
****************
Mol level model
****************
Regression Metrics:
MAE:  1455.0423
RMSE: 2138.2251
R²:   0.9407
****************
Atom level model
*****************
Regression Metrics:
MAE:  2627.7569
RMSE: 3879.8971
R²:   0.8048
energy_U0
****************
Mol level model
****************
Regression Metrics:
MAE:  20.2668
RMSE: 190.4754
R²:   0.9859
****************
Atom level model
*****************
Regression Metrics:
MAE:  137.9587
RMSE: 734.6078
R²:   0.7909
